In [ ]:
import os
import sqlite3
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# MySQL 연결 설정
MYSQL_USER = ""
MYSQL_PASSWORD = ""
MYSQL_HOST = "127.0.0.1"
MYSQL_PORT = "3306"
MYSQL_DB = ""

# SQLAlchemy 엔진 생성
engine = create_engine(f"mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB}")

In [ ]:
# SQLite DB 디렉토리
now_dir = os.getcwd()
db_dir = os.path.join(now_dir, "data")

for db_file in os.listdir(db_dir):
    # SQLite DB 연결
    sqlite_path = os.path.join(db_dir, db_file)
    conn = sqlite3.connect(sqlite_path)
    cur = conn.cursor()

    # 테이블 목록 조회
    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cur.fetchall()

    for table in tables:
        table_name = table[0]
        df = pd.read_sql(f"SELECT * FROM {table_name}", conn)

        # MySQL에 INSERT (존재 시 append, 기존 테이블 drop 안 함)
        df.to_sql(table_name, con=engine, if_exists='append', index=False)
        print(f"Migrate {table_name}'s Table")

    conn.close()